In [11]:
from time import time
import string
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [12]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
stopwords1 = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
cleanedtweets=[]
def clean(listoftweets):
    
    listoffiltered=[]
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    for i in range(0,len(listoftweets)):
         
         
         listoftweets[i]=listoftweets[i].replace('@USER', '') #Remove mentions (@USER)
         
         listoftweets[i]=listoftweets[i].replace('URL', '')
         listoftweets[i]=listoftweets[i].replace('&amp', 'and')
         listoftweets[i]=listoftweets[i].replace('&lt', '')
         listoftweets[i]=listoftweets[i].replace('&gt', '')
         listoftweets[i]=listoftweets[i].replace('\d+', '')
         listoftweets[i] = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          (listoftweets[i])) 
         listoftweets[i]=emoji_pattern.sub(r'', listoftweets[i])
         listoftweets[i]=listoftweets[i].translate(str.maketrans('', '', string.punctuation))
         listoftweets[i]=listoftweets[i].strip()
         listoftweets[i] = re.sub(' +', ' ', listoftweets[i])
         listoftweets[i]=listoftweets[i].lower()
         cleanedtweets.append(listoftweets[i])
         word_tokens = word_tokenize(listoftweets[i])
         filteredsen = [w for w in word_tokens if w not in stopwords1 and w != ' ']
         listoffiltered.append(filteredsen)
    return listoffiltered

In [14]:
gh=[]
df=pd.read_csv("/content/olid-training-v1.0.tsv",sep='\t')
label = df['subtask_a']
columns = ['id','subtask_b','subtask_c']
df.drop(columns, axis=1, inplace=True)
listoftweets = []
for i in range(0,len(df)):
     listoftweets.append(df.iloc[i,0])
gh=clean(listoftweets)

In [15]:
listoftweets[0]

'she should ask a few native americans what their take on this is'

In [132]:
df.head(4)

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF


In [16]:
fulldata=pd.read_csv("/content/olid-training-v1.0.tsv",sep='\t')
columns = ['id','subtask_b','subtask_c']
fulldata.drop(columns, axis=1, inplace=True)
data_off = fulldata[fulldata['subtask_a'] == 'OFF']
data_nff = fulldata[fulldata['subtask_a'] == 'NOT']
listoftweets2 = []
listoftweets3 = []
for i in range(0,len(data_off)):
      listoftweets2.append(data_off.iloc[i,0])
for i in range(0,len(data_nff)):
       listoftweets3.append(data_nff.iloc[i,0])

list2 = clean(listoftweets2)
list3 = clean(listoftweets3)      

In [ ]:
# #return the list of words learned
# learned_words = list(model.wv.vocab)
# #print the learned words
# print(learned_words)

In [ ]:
# top10words = (model.wv.index2word[:10])
# documentmatrix = np.zeros((10, 100))
# for jk in range(0,10):
#    documentmatrix[jk] = model.wv[top10words[jk]]
# documentmatrix[0]

In [ ]:
# flattenlist1=[]
# for i in range(len(documentmatrix)):
#     sum=0
#     for j in range(len(documentmatrix[i])):
#             sum+=documentmatrix[i][j] 
#     flattenlist1.append(sum)        

In [ ]:
# len(flattenlist1)

In [135]:
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from numpy import dot
from numpy.linalg import norm
import re
flattenlist1=[]
flattenlist2=[]
docsim1 = []
docsim2 = []
w2csorted = []
w2csorted1 = []    
firstlist=[]
secondlist = []
top10words =[]
top10words1 = []
file_one1 = "/content/olid-training-v1.0.tsv"
file_one2 = "/content/olid-training-v1.0.tsv"
def compare_texts_word2vec(file_one, file_two, k):
        fulldata=pd.read_csv(file_one1,sep='\t')
        columns = ['id','subtask_b','subtask_c']
        fulldata.drop(columns, axis=1, inplace=True)
        data_off = fulldata[fulldata['subtask_a'] == 'OFF']
        data_nff = fulldata[fulldata['subtask_a'] == 'NOT']
        listoftweets2 = []
        listoftweets3 = []
        for i in range(0,len(data_off)):
             listoftweets2.append(data_off.iloc[i,0])
        for i in range(0,len(data_nff)):
             listoftweets3.append(data_nff.iloc[i,0])

        list2 = clean(listoftweets2)
        list3 = clean(listoftweets3)      
        model = Word2Vec(list2, min_count=5)
        model1 = Word2Vec(list3, min_count=5)
        #return the list of words learned
        learned_words = list(model.wv.vocab)
        learned_words1 = list(model1.wv.vocab)
        w2c = dict()
        w2c1=dict()
        for item in model.wv.vocab:
           w2c[item]=model.wv.vocab[item].count
           w2csorted = (sorted(w2c.items(),key=lambda x: x[1], reverse=True))
        for item in model1.wv.vocab:
           w2c1[item]=model1.wv.vocab[item].count
           w2csorted1 = (sorted(w2c1.items(),key=lambda x: x[1], reverse=True))
        for first, second in w2csorted:
               firstlist.append(first)
        for  first1 ,second1 in w2csorted1:
             secondlist.append(first1)       
        top10words = firstlist[0:k]
        top10words1 = secondlist[0:k]
        print(top10words)
        print(top10words1)
        documentmatrix = np.zeros((k, 100))
        documentmatrix1 = np.zeros((k, 100))
           
        for jk in range(0,k):
             documentmatrix[jk] = model.wv[top10words[jk]]
             documentmatrix1[jk] = model1.wv[top10words1[jk]]
             my_dicts = model.wv.most_similar(top10words[jk],topn=10)
             my_dicts1 = model1.wv.most_similar(top10words[jk],topn=10)
             listgh=[]
             listfg=[] 
             for first, second in my_dicts:
                  listgh.append(first)
             for first1, second1 in my_dicts1:
                  listfg.append(first1)
             docsim1.append(listgh)
             docsim2.append(listfg)
        print(docsim1)
        print(docsim2)
        for i in range(len(documentmatrix)):
            sum=0
            sum1=0
            for j in range(len(documentmatrix[i])):
                 sum+=documentmatrix[i][j] 
                 sum1+=documentmatrix1[i][j]
            flattenlist1.append(sum)
            flattenlist2.append(sum1)
            print(len(flattenlist1))
            print(len(flattenlist2))
       
        cos_sim = dot(flattenlist1, flattenlist2)/(norm(flattenlist1)*norm(flattenlist2))
        return cos_sim

In [ ]:
cosvalue = compare_texts_word2vec(file_one1, file_one2, 10)
cosvalue

['liberals', 'gun', 'like', 'control', 'people', 'antifa', 'shit', 'maga', 'conservatives', 'trump']
['liberals', 'antifa', 'gun', 'control', 'maga', 'conservatives', 'like', 'people', 'trump', 'one']
[['conservatives', 'trump', 'like', 'women', 'maga', 'way', 'left', 'could', 'antifa', 'man'], ['laws', 'need', 'country', 'nothing', 'would', 'people', 'want', 'go', 'also', 'us'], ['antifa', 'conservatives', 'get', 'going', 'liberals', 'time', 'people', 'know', 'president', 'us'], ['laws', 'law', 'also', 'violence', 'right', 'liberals', 'want', 'nothing', 'country', 'stop'], ['like', 'know', 'trump', 'time', 'nothing', 'country', 'us', 'go', 'get', 'shit'], ['like', 'time', 'conservatives', 'get', 'country', 'still', 'going', 'liberals', 'never', 'could'], ['like', 'conservatives', 'trump', 'maga', 'antifa', 'left', 'people', 'never', 'going', 'liberals'], ['conservatives', 'liberals', 'us', 'get', 'trump', 'women', 'shit', 'like', 'could', 'going'], ['trump', 'liberals', 'like', 'maga'

-0.9919267062728845

In [136]:
cosvalue = compare_texts_word2vec(file_one1, file_one2, 5)
cosvalue

['liberals', 'gun', 'like', 'control', 'people']
['liberals', 'antifa', 'gun', 'control', 'maga']
[['like', 'conservatives', 'women', 'would', 'country', 'going', 'people', 'one', 'get', 'still'], ['country', 'laws', 'go', 'need', 'like', 'away', 'people', 'maga', 'want', 'good'], ['liberals', 'one', 'people', 'antifa', 'still', 'conservatives', 'democrats', 'could', 'left', 'get'], ['laws', 'every', 'want', 'support', 'like', 'great', 'us', 'liberals', 'people', 'many'], ['like', 'democrats', 'liberals', 'antifa', 'get', 'also', 'say', 'let', 'believe', 'see']]
[['get', 'liberal', 'party', 'going', 'got', 'think', 'conservatives', 'still', 'say', 'used'], ['laws', 'strict', 'chicago', 'guns', 'sensible', 'common', 'control', 'nra', 'california', 'rights'], ['still', 'one', 'people', 'also', 'nothing', 'even', 'could', 'would', 'really', 'liberals'], ['laws', 'strict', 'gun', 'guns', 'sensible', 'chicago', 'nra', 'safety', 'support', 'california'], ['also', 'democrats', 'new', 'see', '

0.9883611747038017

In [149]:
cosvalue = compare_texts_word2vec(file_one1, file_one2, 20)
cosvalue

['liberals', 'gun', 'like', 'control', 'people', 'antifa', 'shit', 'maga', 'conservatives', 'trump', 'get', 'know', 'one', 'think', 'go', 'fuck', 'would', 'ass', 'good', 'right']
['liberals', 'antifa', 'gun', 'control', 'maga', 'conservatives', 'like', 'people', 'trump', 'one', 'get', 'know', 'think', 'would', 'right', 'good', 'us', 'time', 'love', 'want']
[['like', 'conservatives', 'women', 'would', 'country', 'going', 'people', 'one', 'get', 'still'], ['country', 'laws', 'go', 'need', 'like', 'away', 'people', 'maga', 'want', 'good'], ['liberals', 'one', 'people', 'antifa', 'still', 'conservatives', 'democrats', 'could', 'left', 'get'], ['laws', 'every', 'want', 'support', 'like', 'great', 'us', 'liberals', 'people', 'many'], ['like', 'democrats', 'liberals', 'antifa', 'get', 'also', 'say', 'let', 'believe', 'see'], ['like', 'get', 'one', 'think', 'women', 'country', 'conservatives', 'liberal', 'antifa', 'people'], ['country', 'like', 'maga', 'go', 'laws', 'need', 'make', 'away', 'go

0.9685318076084142

In [ ]:
type(w2csorted)

list

In [ ]:
w2csorted

[]

In [150]:
cosvalue

0.9685318076084142

In [151]:
docsim1

[['like',
  'conservatives',
  'women',
  'would',
  'country',
  'going',
  'people',
  'one',
  'get',
  'still'],
 ['country',
  'laws',
  'go',
  'need',
  'like',
  'away',
  'people',
  'maga',
  'want',
  'good'],
 ['liberals',
  'one',
  'people',
  'antifa',
  'still',
  'conservatives',
  'democrats',
  'could',
  'left',
  'get'],
 ['laws',
  'every',
  'want',
  'support',
  'like',
  'great',
  'us',
  'liberals',
  'people',
  'many'],
 ['like',
  'democrats',
  'liberals',
  'antifa',
  'get',
  'also',
  'say',
  'let',
  'believe',
  'see'],
 ['like',
  'get',
  'one',
  'think',
  'women',
  'country',
  'conservatives',
  'liberal',
  'antifa',
  'people'],
 ['country',
  'like',
  'maga',
  'go',
  'laws',
  'need',
  'make',
  'away',
  'good',
  'people'],
 ['liberals',
  'one',
  'antifa',
  'get',
  'people',
  'still',
  'left',
  'could',
  'conservatives',
  'said'],
 ['laws',
  'people',
  'want',
  'stupid',
  'stop',
  'every',
  'like',
  'great',
  'noth

In [152]:
docsim2

[['get',
  'liberal',
  'party',
  'going',
  'got',
  'think',
  'conservatives',
  'still',
  'say',
  'used'],
 ['laws',
  'strict',
  'chicago',
  'guns',
  'sensible',
  'common',
  'control',
  'nra',
  'california',
  'rights'],
 ['still',
  'one',
  'people',
  'also',
  'nothing',
  'even',
  'could',
  'would',
  'really',
  'liberals'],
 ['laws',
  'strict',
  'gun',
  'guns',
  'sensible',
  'chicago',
  'nra',
  'safety',
  'support',
  'california'],
 ['also',
  'democrats',
  'new',
  'see',
  'work',
  'government',
  'big',
  'liberals',
  'nothing',
  'saying'],
 ['liberal',
  'party',
  'conservatives',
  'going',
  'got',
  'think',
  'used',
  'still',
  'trying',
  'would'],
 ['laws',
  'strict',
  'sensible',
  'guns',
  'nra',
  'chicago',
  'control',
  'shooting',
  'california',
  'support'],
 ['one',
  'still',
  'people',
  'also',
  'liberals',
  'show',
  'nothing',
  'got',
  'would',
  'use'],
 ['laws',
  'strict',
  'sensible',
  'chicago',
  'gun',
  

In [153]:
flat1=set().union(*docsim1)
flat2=set().union(*docsim2)
def common_elements(list1, list2):
        #flat1 = [x for sublist in list1 for x in sublist]
        #flat2 = [x1 for sublist1 in list2 for x1 in sublist1]
        # flat1 = set().union(*list1)
        # flat2 = set().union(*list2)
        # print(flat1)
        # print(flat2)
        return [element for element in list1 if element in list2]

In [154]:
result = common_elements(flat1,flat2)

In [142]:
result

['support',
 'still',
 'get',
 'see',
 'going',
 'people',
 'democrats',
 'conservatives',
 'could',
 'laws',
 'also',
 'liberals',
 'one',
 'would',
 'say']

In [155]:
overlapping = set(result)

In [156]:
overlapping

{'also',
 'bad',
 'believe',
 'conservatives',
 'could',
 'democrats',
 'even',
 'every',
 'get',
 'go',
 'going',
 'good',
 'got',
 'laws',
 'left',
 'let',
 'liberal',
 'liberals',
 'maga',
 'make',
 'man',
 'many',
 'never',
 'nothing',
 'one',
 'people',
 'president',
 'said',
 'say',
 'see',
 'still',
 'support',
 'think',
 'time',
 'trump',
 'us',
 'would'}

In [ ]:
flat1

{'also',
 'antifa',
 'conservatives',
 'could',
 'country',
 'get',
 'go',
 'going',
 'know',
 'law',
 'laws',
 'left',
 'liberals',
 'like',
 'maga',
 'man',
 'need',
 'never',
 'nothing',
 'one',
 'people',
 'president',
 'right',
 'shit',
 'still',
 'stop',
 'time',
 'trump',
 'us',
 'violence',
 'want',
 'way',
 'women',
 'would'}

In [157]:
uniquelist1 = (flat1.difference(overlapping))

In [158]:
uniquelist2 = (flat2.difference(overlapping))

In [159]:
print(uniquelist1)

{'want', 'anyone', 'antifa', 'away', 'stupid', 'need', 'much', 'know', 'country', 'back', 'stop', 'women', 'like', 'great'}


In [160]:
print(uniquelist2)

{'new', 'chicago', 'business', 'come', 'walkaway', 'always', 'thought', 'children', 'true', 'rights', 'making', 'qanon', 'yet', 'anything', 'really', 'guns', 'show', 'follow', 'big', 'government', 'someone', 'america', 'right', 'sensible', 'woman', 'saying', 'kag', 'used', 'strict', 'since', 'first', 'strong', 'party', 'kavanaugh', 'another', 'work', 'nra', 'shooting', 'home', 'made', 'long', 'strictest', 'gun', 'control', 'california', 'everyone', 'thank', 'usa', 'thing', 'safety', 'tcot', 'use', 'twitter', 'well', 'best', 'today', 'trying', 'problem', 'soros', 'common'}


In [ ]:
# from keras_preprocessing.text import Tokenizer
# from keras_preprocessing.sequence import pad_sequences
# tokenizer=Tokenizer()
# tokenizer.fit_on_texts(gh)
# tokenized_documents=tokenizer.texts_to_sequences(gh)
# tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
# vocab_size=len(tokenizer.word_index)+1
# print (tokenized_paded_documents[0])
# labels=df['subtask_a']
# from numpy import array
# from keras_preprocessing.text import one_hot
# from keras_preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Flatten
# from tensorflow.keras.layers import Embedding
# max_length=64

In [8]:
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
#label1 = le.fit_transform(df['subtask_a'])


In [87]:
from nltk.metrics.segmentation import ghd
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
t = Tokenizer()

vocab_size=0
embeddings_index12 = dict()

def train_MLP_model1(path_to_train_file, num_layers = 2):
			df=pd.read_csv(path_to_train_file, sep='\t')
			label = df['subtask_a']
			label1 = le.fit_transform(df['subtask_a'])
			columns = ['id','subtask_b','subtask_c']
			df.drop(columns, axis=1, inplace=True)
			listoftweets = []
			for i in range(0,len(df)):
						listoftweets.append(df.iloc[i,0])
			gh=clean(listoftweets)
			print(t.fit_on_texts(gh))
			vocab_size = len(t.word_index) + 1
			# integer encode the documents
			encoded_docs = t.texts_to_sequences(gh)
			print(encoded_docs)
			# pad documents to a max length of 4 words
			max_length = 64
			padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
			print(padded_docs)
			with open('glove.6B.100d.txt') as file:
						for line in file:
							values = line.split()
							word = values[0]
							coefs = np.asarray(values[1:], dtype='float32')
							embeddings_index[word] = coefs
			
			print('Loaded %s word vectors.' % len(embeddings_index))
			# create a weight matrix for words in training docs
			embedding_matrix = zeros((vocab_size, 100))
			for word, i in t.word_index.items():
				embedding_vector = embeddings_index.get(word)
				if embedding_vector is not None:
						embedding_matrix[i] = embedding_vector
			# define model
			model = Sequential()
			e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=64, trainable=False)
			model.add(e)
			model.add(Flatten())
			model.add(Dense(1, activation='sigmoid'))
			# compile the model
			model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
			# summarize the model
			print(model.summary())
			# fit the model
			model.fit(padded_docs, label1, epochs=50, verbose=0)
			# evaluate the model
			loss, accuracy = model.evaluate(padded_docs, label1, verbose=0)
			print('Accuracy: %f' % (accuracy*100))
			return model

In [88]:
jk = train_MLP_model1("/content/olid-training-v1.0.tsv", num_layers = 1)

None
[[250, 2457, 180, 43], [19, 201, 817, 6, 9], [3430, 6101, 1704, 2252, 1617, 4825, 1282, 1705, 45, 8701, 185, 3431, 3432, 4826, 3430, 6, 208, 707, 389], [95, 8702, 534, 22, 8703], [125, 351, 1, 1099, 454, 380, 519], [1, 8704], [99, 8705, 1216, 22], [408, 157, 61, 560, 667, 5, 561, 4827, 843, 187, 1788, 17, 954, 463, 5, 2, 3, 2253], [562, 8706], [352, 23, 58, 6102, 140, 138, 8707, 1, 627, 1618, 52, 48, 8708, 6103], [685, 78, 2, 3], [708, 49, 11, 256, 2254, 3433, 132, 100, 230, 60, 225, 25, 879, 6104, 8709, 8710, 4021, 1706, 8711, 993, 6], [147, 409, 1217, 543, 210, 297, 616, 209, 4022], [1619, 77, 136, 361, 6105, 3434, 6106], [5, 8712, 5, 5, 6107, 163, 1513], [42, 1618, 4023], [6, 4024, 5, 11, 1100, 28, 5, 24, 430, 38, 211, 11, 369, 390, 1, 1926], [18, 15, 276, 1789, 2070, 1283, 2255, 2458, 1283, 78, 1514, 2, 3, 16, 202, 1620, 1054, 6108], [1707, 8713, 668, 106, 8714, 709, 787, 5, 16, 114, 111], [2, 3, 287, 464], [61, 1358, 399, 710, 844, 8715, 49, 6, 8716], [447], [214, 5, 7, 734, 

In [107]:
from nltk.metrics.segmentation import ghd
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
t2 = Tokenizer()

vocab_size=0
#embeddings_index12 = dict()

def test_MLP_model1(path_to_test_file, model):
      df1=pd.read_csv(path_to_test_file, sep='\t')
      label12= pd.read_csv("/content/labels-levela.csv", sep='\t',header=None)
      label13 = le.fit_transform(label12)
      columns = ['id']
      df1.drop(columns, axis=1, inplace=True)
   
      listoftweets = []
      for i in range(0,len(df1)):
            listoftweets.append(df1.iloc[i,0])
      gh=clean(listoftweets)
      print(t2.fit_on_texts(gh))
      vocab_size = len(t2.word_index) + 1
      # integer encode the documents
      encoded_docs = t2.texts_to_sequences(gh)
      print(encoded_docs)
      # pad documents to a max length of 4 words
      max_length = 64
      padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
      print(padded_docs)
      # with open('glove.6B.100d.txt') as file:
      # 			for line in file:
      # 				values = line.split()
      # 				word = values[0]
      # 				coefs = np.asarray(values[1:], dtype='float32')
      # 				embeddings_index[word] = coefs
      
      print('Loaded %s word vectors.' % len(embeddings_index))
      # create a weight matrix for words in training docs
      embedding_matrix = zeros((vocab_size, 100))
      for word, i in t2.word_index.items():
        if word in embeddings_index12.keys():
            embedding_vector = embeddings_index12.get(word)
            if embedding_vector is not None:
               embedding_matrix[i] = embedding_vector
      # define model
      # model = Sequential()
      # e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=44, trainable=False)
      # model.add(e)
      # model.add(Flatten())
      # model.add(Dense(1, activation='sigmoid'))
      # # compile the model
      # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
      # # summarize the model
      print(model.summary())
      # fit the model
      model.fit(padded_docs, label13, epochs=50, verbose=0)
      # evaluate the model
      loss, accuracy = model.evaluate(padded_docs, label13, verbose=0)
      print('Accuracy: %f' % (accuracy*100))
      

In [108]:
test_MLP_model1("/content/testset-levela.tsv", jk)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


None
[[1677, 1678, 1679, 1680, 21, 28, 3, 441, 1681, 1682, 913, 1683, 253, 1684, 1685, 593, 442, 254, 443, 1686, 914, 915, 123, 124, 40], [594, 1687, 1, 1688, 323, 444, 21, 14, 93], [916, 595, 7, 255, 11, 1689, 1690, 1691, 596, 1692, 1693, 324, 2, 1694, 60, 1695, 256], [203, 1696, 112, 70, 32, 1697, 29, 94, 597, 1698, 1699, 1700], [1701, 917, 918, 1702, 204, 41, 919, 3, 1703, 172, 172], [140, 26], [920, 921, 325, 445, 1704, 922, 1705, 923, 598, 599, 257, 924, 6, 1706, 600, 601, 173, 27, 602, 1707, 1708, 8, 603, 1709, 604], [8, 605, 1710, 326, 1711, 327], [446, 1712, 606, 925, 607, 1713, 1714], [608, 1715, 1716, 18, 60, 447, 258], [1, 52, 448, 1717, 609, 449], [1718, 53, 259, 174, 926, 927, 928, 929, 205, 930, 1719, 931, 1720, 932, 206, 175, 933, 934, 38, 26, 935, 1721], [1722, 328, 450, 1], [141, 40, 329, 123, 124, 451, 330, 19, 331, 113, 95], [452, 1723, 936, 1724], [937, 453, 6, 610, 71], [1725, 142, 260, 143, 207, 1726, 1727, 176, 1728, 1729, 18, 938, 939, 125, 8, 1730, 939, 1731, 1

In [53]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import math
import pandas as pd
gh=[]
tokenizer=Tokenizer()
surbhi = dict()
vocab_size=0
embeddings_index = dict()
def train_MLP_model(path_to_train_file, num_layers = 2):
   df=pd.read_csv(path_to_train_file, sep='\t')
   label = df['subtask_a']
   label1 = le.fit_transform(df['subtask_a'])
   columns = ['id','subtask_b','subtask_c']
   df.drop(columns, axis=1, inplace=True)
   listoftweets = []
   for i in range(0,len(df)):
        listoftweets.append(df.iloc[i,0])
   gh=clean(listoftweets)
  
   tokenizer.fit_on_texts(gh)
   tokenized_documents=tokenizer.texts_to_sequences(gh)
   tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
   print(tokenizer.word_index)
    
   vocab_size=len(tokenizer.word_index)+1
  
   print (tokenized_paded_documents[0])
   #reading Glove word embeddings into a dictionary with "word" as key and values as word vectors
  #  embeddings_index = dict()

   with open('glove.6B.100d.txt') as file:
        for line in file:
           values = line.split()
           word = values[0]
           coefs = np.asarray(values[1:], dtype='float32')
           embeddings_index[word] = coefs
            
   # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
   embedding_matrix=np.zeros((vocab_size,100))
  #  for  hj in range(0,len(tokenized_paded_documents)):
  #        for tv in range(0,len(tokenized_paded_documents[hj])):
  #             if tokenized_paded_documents[hj][tv] in tokenizer.word_index.items
   for word,i in tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
    #print(embedding_vector)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
   document_embeddings=np.zeros((len(tokenized_paded_documents),100))

   dict_keys=list(tokenizer.word_index.keys())
   dict_values=list(tokenizer.word_index.values())
   for i in range(len(tokenized_paded_documents)):
      joh = np.zeros((1,100),  dtype = np.float32)
      for j in range(len(tokenized_paded_documents[0])):
            print(tokenized_paded_documents[i][j])
            if tokenized_paded_documents[i][j] != 0:
               val_index=dict_values.index(tokenized_paded_documents[i][j])
               print("Associated key is:")
               myKey=dict_keys[val_index]
               print(myKey)
               if embeddings_index.get(myKey) is None:
                   continue
               surbhi[myKey]=embeddings_index.get(myKey)    
               print("test devesh",embeddings_index.get(myKey))
               
               #print(embeddings_index.get(myKey).dtype)
               joh += embeddings_index.get(myKey)
      document_embeddings[i]=joh

      document_embeddings.shape
   for i in range(0,len(document_embeddings)):
     for j in range(0,len(document_embeddings[i])):
           if(math.isnan(document_embeddings[i][j])):
                      document_embeddings[i][j]=0;        
 
   X_train, X_test, y_train, y_test = train_test_split(document_embeddings, label1, stratify=label1,
                                                     random_state=1)
   if num_layers==1:
     clf = MLPClassifier(hidden_layer_sizes=(9), activation='logistic', solver='adam', max_iter=2000)
   elif num_layers==2:
     clf = MLPClassifier(hidden_layer_sizes=(9,12), activation='logistic', solver='adam', max_iter=2000)
   elif num_layers==3:
     clf = MLPClassifier(hidden_layer_sizes=(9,12, 15), activation='logistic', solver='adam', max_iter=2000)   
   elif num_layers==4:
      clf = MLPClassifier(hidden_layer_sizes=(9,12, 15,1), activation='logistic', solver='adam', max_iter=2000)   
   
   hn = clf.fit(X_train, y_train)
   print(hn.predict_proba(X_test))
   predicted_values = hn.predict(X_test)
   print(confusion_matrix(y_test,predicted_values))
   print(classification_report(y_test,predicted_values))          
   return hn

In [54]:
bn = train_MLP_model("/content/olid-training-v1.0.tsv", num_layers = 2)

Streaming output truncated to the last 5000 lines.
  0.39152   0.124    -0.20873  -0.18473  -0.56184   0.55693   0.012114
 -0.54545  -0.31409   0.1       0.31543   0.74757  -0.47734  -0.18332
 -0.65623   0.40768  -0.30697  -0.47247  -0.7421   -0.44978  -0.078122
 -0.52673  -0.70633   1.3271    0.26298  -0.91      0.91632  -0.51643
  0.20284  -0.25402  -1.2566    0.20271   0.92105  -0.57574  -0.15105
 -0.24831   0.36673  -0.53987   0.18534   0.25713   0.38794  -0.54137
  0.67817  -0.17251 ]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
Associated key is:
right
test devesh [ 1.0060e-01  2.0405e-01  4.6027e-01 -6.1584e-01 -3.1460e-01  5.9874e-01
 -5.1974e-01 -3.6454e-01 -5.7500e-02  1.1387e-02  5.9460e-02  1.0890e-01
  6.9055e-01 -9.6085e-02  2.5253e-01 -1.6050e-01 -3.2447e-01  2.8349e-01
 -5.4160e-01  1.1315e-03  5.9300e-01  4.8945e-01 -6.3187e-02 -1.6199e-01
  7.5422e-01  1.8403e-01 -4.4059e-01 -9.2566e-01  6.3593e-01 -2

In [76]:
bn2 = train_MLP_model("/content/olid-training-v1.0.tsv", num_layers = 1)

Streaming output truncated to the last 5000 lines.
  0.39152   0.124    -0.20873  -0.18473  -0.56184   0.55693   0.012114
 -0.54545  -0.31409   0.1       0.31543   0.74757  -0.47734  -0.18332
 -0.65623   0.40768  -0.30697  -0.47247  -0.7421   -0.44978  -0.078122
 -0.52673  -0.70633   1.3271    0.26298  -0.91      0.91632  -0.51643
  0.20284  -0.25402  -1.2566    0.20271   0.92105  -0.57574  -0.15105
 -0.24831   0.36673  -0.53987   0.18534   0.25713   0.38794  -0.54137
  0.67817  -0.17251 ]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
Associated key is:
right
test devesh [ 1.0060e-01  2.0405e-01  4.6027e-01 -6.1584e-01 -3.1460e-01  5.9874e-01
 -5.1974e-01 -3.6454e-01 -5.7500e-02  1.1387e-02  5.9460e-02  1.0890e-01
  6.9055e-01 -9.6085e-02  2.5253e-01 -1.6050e-01 -3.2447e-01  2.8349e-01
 -5.4160e-01  1.1315e-03  5.9300e-01  4.8945e-01 -6.3187e-02 -1.6199e-01
  7.5422e-01  1.8403e-01 -4.4059e-01 -9.2566e-01  6.3593e-01 -2

In [79]:
bn3 = train_MLP_model("/content/olid-training-v1.0.tsv", num_layers = 3)

Streaming output truncated to the last 5000 lines.
  0.39152   0.124    -0.20873  -0.18473  -0.56184   0.55693   0.012114
 -0.54545  -0.31409   0.1       0.31543   0.74757  -0.47734  -0.18332
 -0.65623   0.40768  -0.30697  -0.47247  -0.7421   -0.44978  -0.078122
 -0.52673  -0.70633   1.3271    0.26298  -0.91      0.91632  -0.51643
  0.20284  -0.25402  -1.2566    0.20271   0.92105  -0.57574  -0.15105
 -0.24831   0.36673  -0.53987   0.18534   0.25713   0.38794  -0.54137
  0.67817  -0.17251 ]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
Associated key is:
right
test devesh [ 1.0060e-01  2.0405e-01  4.6027e-01 -6.1584e-01 -3.1460e-01  5.9874e-01
 -5.1974e-01 -3.6454e-01 -5.7500e-02  1.1387e-02  5.9460e-02  1.0890e-01
  6.9055e-01 -9.6085e-02  2.5253e-01 -1.6050e-01 -3.2447e-01  2.8349e-01
 -5.4160e-01  1.1315e-03  5.9300e-01  4.8945e-01 -6.3187e-02 -1.6199e-01
  7.5422e-01  1.8403e-01 -4.4059e-01 -9.2566e-01  6.3593e-01 -2

In [68]:

def test_MLP_model(path_to_test_file, MLP_model):
   df1=pd.read_csv(path_to_test_file, sep='\t')
   label12= pd.read_csv("/content/labels-levela.csv", sep='\t',header=None)
   label13 = le.fit_transform(label12)
   columns = ['id']
   df1.drop(columns, axis=1, inplace=True)
   listoftweetstest = []
   hm=[]
   for i in range(0,len(df1)):
        listoftweetstest.append(df1.iloc[i,0])
   hm=clean(listoftweetstest)
   tokenizer1=Tokenizer()
   tokenizer1.fit_on_texts(hm)
   tokenized_documents1=tokenizer1.texts_to_sequences(hm)
   tokenized_paded_documents1=pad_sequences(tokenized_documents1,maxlen=64,padding='post')
   print(tokenizer1.word_index)
   vocab_size=len(tokenizer1.word_index)+1
   print (tokenized_paded_documents1[0])
    
  #  # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
   embedding_matrix=np.zeros((vocab_size,100))

  #  for word,i in tokenizer.word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #   #print(embedding_vector)
  #     if embedding_vector is not None:
  #         embedding_matrix[i] = embedding_vector

   document_embeddings=np.zeros((len(tokenized_paded_documents1),100))
   dict_keys=list(tokenizer.word_index.keys())
   dict_values=list(tokenizer.word_index.values())
   for i in range(len(tokenized_paded_documents1)):
      joh1 = np.zeros((1,100),  dtype = np.float32)
      for j in range(len(tokenized_paded_documents1[0])):
            print(tokenized_paded_documents1[i][j])
            if tokenized_paded_documents1[i][j] != 0: 
                if tokenized_paded_documents1[i][j] in dict_values :
                   val_index=dict_values.index(tokenized_paded_documents1[i][j])
                   print("Associated key is:")
                   myKey=dict_keys[val_index]
                   print(myKey)
                   if myKey in surbhi.keys():
                      joh1 += surbhi[myKey]
               
                   
      document_embeddings[i]=joh1
   for i in range(0,len(document_embeddings)):
     for j in range(0,len(document_embeddings[i])):
           if(math.isnan(document_embeddings[i][j])):
                      document_embeddings[i][j]=0;        
 
  
   cv = MLP_model.predict_proba(document_embeddings)
   print(cv)
   predicted_values = MLP_model.predict(document_embeddings)
   print(confusion_matrix(label13,predicted_values))
   print(classification_report(label13,predicted_values))  
   df3 = pd.DataFrame()
   df3["prob"]=cv[:,0]   
   df3["output"]=predicted_values
   df3.to_csv('testmlp.csv', sep='\t', index=False)      
   

In [69]:
test_MLP_model("/content/testset-levela.tsv", bn)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Streaming output truncated to the last 5000 lines.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4710
Associated key is:
waging
672
Associated key is:
system
702
Associated key is:
super
81
Associated key is:
work
331
Associated key is:
deal
4711
Associated key is:
awards
33
Associated key is:
still
4712
Associated key is:
golden
4713
Associated key is:
pivot
6
Associated key is:
maga
1611
Associated key is:
argue
4714
Associated key is:
mail
1530
Associated key is:
felt
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1224
Associated key is:
david
11
Associated key is:
one
319
Associated key is:
sad
1614
Associated key is:
arrested
4715
Associated key is:
repeating
4716
Associated key is:
hosting
196
Associated key is:
yeah
19
Associated key is:
go
582
Associated key is:
college
383
Associated key is:
happened
4717
Associated key is:
achievements
7
Associated key is:
conservatives
21
Associated key is:
see
2
Asso

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
test_MLP_model("/content/testset-levela.tsv", bn3)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Streaming output truncated to the last 5000 lines.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4710
Associated key is:
waging
672
Associated key is:
system
702
Associated key is:
super
81
Associated key is:
work
331
Associated key is:
deal
4711
Associated key is:
awards
33
Associated key is:
still
4712
Associated key is:
golden
4713
Associated key is:
pivot
6
Associated key is:
maga
1611
Associated key is:
argue
4714
Associated key is:
mail
1530
Associated key is:
felt
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1224
Associated key is:
david
11
Associated key is:
one
319
Associated key is:
sad
1614
Associated key is:
arrested
4715
Associated key is:
repeating
4716
Associated key is:
hosting
196
Associated key is:
yeah
19
Associated key is:
go
582
Associated key is:
college
383
Associated key is:
happened
4717
Associated key is:
achievements
7
Associated key is:
conservatives
21
Associated key is:
see
2
Asso

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))
glove_vectors = gensim.downloader.load('glove-twitter-25')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[==================================================] 100.0% 104.8/104.8MB downloaded


In [126]:
from gensim.models import Word2Vec
med=dict()
gh=[]
def update_embeddings(path_to_train_file):
   df2=pd.read_csv(path_to_train_file, sep='\t')
   label = df2['subtask_a']
   label1 = le.fit_transform(df2['subtask_a'])
   columns = ['id','subtask_b','subtask_c']
   df2.drop(columns, axis=1, inplace=True)
   listoftweets = []
   for i in range(0,len(df2)):
        listoftweets.append(df2.iloc[i,0])
   gh=clean(listoftweets)
  
   #for gh in listoftweets:
   model = Word2Vec(sentences=gh, size=100, window=5, min_count=1, workers=4)
   op = model.save("word2vec.model")
   print(op)
   med = Word2Vec.load("word2vec.model")
   print(med)
   model.train(gh, total_examples=1, epochs=1)
   return op, med

In [127]:
op, med = update_embeddings("/content/olid-training-v1.0.tsv")

None
Word2Vec(vocab=18436, size=100, alpha=0.025)


In [5]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = med.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

#vector = wv['computer']  # Get numpy vector of a word

NameError: ignored

In [9]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import math
import pandas as pd
gh=[]
tokenizer=Tokenizer()
surbhi12 = dict()
vocab_size=0
embeddings_index = dict()
def train_MLP_model(path_to_train_file, med, num_layers = 2):
   df=pd.read_csv(path_to_train_file, sep='\t')
   label = df['subtask_a']
   label1 = le.fit_transform(df['subtask_a'])
   columns = ['id','subtask_b','subtask_c']
   df.drop(columns, axis=1, inplace=True)
   listoftweets = []
   for i in range(0,len(df)):
        listoftweets.append(df.iloc[i,0])
   gh=clean(listoftweets)
  
   tokenizer.fit_on_texts(gh)
   tokenized_documents=tokenizer.texts_to_sequences(gh)
   tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
   print(tokenizer.word_index)
    
   vocab_size=len(tokenizer.word_index)+1
  
   print (tokenized_paded_documents[0])
   #reading Glove word embeddings into a dictionary with "word" as key and values as word vectors
  #  embeddings_index = dict()

  #  with open('glove.6B.100d.txt') as file:
  #       for line in file:
  #          values = line.split()
  #          word = values[0]
  #          coefs = np.asarray(values[1:], dtype='float32')
  #          embeddings_index[word] = coefs
            
   # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
   embedding_matrix=np.zeros((vocab_size,100))
  #  for  hj in range(0,len(tokenized_paded_documents)):
  #        for tv in range(0,len(tokenized_paded_documents[hj])):
  #             if tokenized_paded_documents[hj][tv] in tokenizer.word_index.items
   for word,i in tokenizer.word_index.items():
      if med[word] is not None:
         embedding_vector = med[word]
    #print(embedding_vector)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
   document_embeddings=np.zeros((len(tokenized_paded_documents),100))

   dict_keys=list(tokenizer.word_index.keys())
   dict_values=list(tokenizer.word_index.values())
   for i in range(len(tokenized_paded_documents)):
      joh = np.zeros((1,100),  dtype = np.float32)
      for j in range(len(tokenized_paded_documents[0])):
            print(tokenized_paded_documents[i][j])
            if tokenized_paded_documents[i][j] != 0:
               val_index=dict_values.index(tokenized_paded_documents[i][j])
               print("Associated key is:")
               myKey=dict_keys[val_index]
               print(myKey)
               if med[myKey] is not None:
                  surbhi12[myKey]= med[myKey]    
              # print("test devesh",embeddings_index.get(myKey))
               
               #print(embeddings_index.get(myKey).dtype)
                  joh += med[myKey]
      document_embeddings[i]=joh

      document_embeddings.shape
   for i in range(0,len(document_embeddings)):
     for j in range(0,len(document_embeddings[i])):
           if(math.isnan(document_embeddings[i][j])):
                      document_embeddings[i][j]=0;        
 
   X_train, X_test, y_train, y_test = train_test_split(document_embeddings, label1, stratify=label1,
                                                     random_state=1)
   if num_layers==1:
     clf = MLPClassifier(hidden_layer_sizes=(1), activation='logistic', solver='adam', max_iter=2000)
   elif num_layers==2:
     clf = MLPClassifier(hidden_layer_sizes=(9,1), activation='logistic', solver='adam', max_iter=2000)
   elif num_layers==3:
     clf = MLPClassifier(hidden_layer_sizes=(9,12, 1), activation='logistic', solver='adam', max_iter=2000)   
   elif num_layers==4:
      clf = MLPClassifier(hidden_layer_sizes=(9,12, 15,1), activation='logistic', solver='adam', max_iter=2000)   
   
   hn = clf.fit(X_train, y_train)
   print(hn.predict_proba(X_test))
   predicted_values = hn.predict(X_test)
   print(confusion_matrix(y_test,predicted_values))
   print(classification_report(y_test,predicted_values))          
   return hn

In [17]:
iu = train_MLP_model("/content/olid-training-v1.0.tsv",  wv, num_layers = 2)

Streaming output truncated to the last 5000 lines.
Associated key is:
geter
133
Associated key is:
done
18377
Associated key is:
fnma
18378
Associated key is:
fmcc
6548
Associated key is:
fanniegate
18379
Associated key is:
endcorruption
2333
Associated key is:
lockthemallup
6
Associated key is:
maga
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
55
Associated key is:
keep
147
Associated key is:
lying
8452
Associated key is:
bud
11
Associated key is:
one
1100
Associated key is:
listening
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7812
Associated key is:
jag
2326
Associated key is:
ref
4685
Associated key is:
tony
8698
Associated key is:
corrente
115
Associated key is:
needs
10
Associated key is:
get
428
Associated key is:
nfl
1278
Associated key is:
bias
534
Associated key is:
piece
979
Associated key is:
garbage
121
Associated key is:
made
651
Associated key is:
horrible
98
Associated key

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
def test_MLP_model(path_to_test_file, med, MLP_model):
   df1=pd.read_csv(path_to_test_file, sep='\t')
   label12= pd.read_csv("/content/labels-levela.csv", sep='\t',header=None)
   label13 = le.fit_transform(label12)
   columns = ['id']
   df1.drop(columns, axis=1, inplace=True)
   listoftweetstest = []
   hm=[]
   for i in range(0,len(df1)):
        listoftweetstest.append(df1.iloc[i,0])
   hm=clean(listoftweetstest)
   tokenizer1=Tokenizer()
   tokenizer1.fit_on_texts(hm)
   tokenized_documents1=tokenizer1.texts_to_sequences(hm)
   tokenized_paded_documents1=pad_sequences(tokenized_documents1,maxlen=64,padding='post')
   print(tokenizer1.word_index)
   vocab_size=len(tokenizer1.word_index)+1
   print (tokenized_paded_documents1[0])
    
  #  # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
   embedding_matrix=np.zeros((vocab_size,100))

  #  for word,i in tokenizer1.word_index.items():
  #     if med[word] is not None:
  #        embedding_vector = med[word]
  #   #print(embedding_vector)
  #     if embedding_vector is not None:
  #         embedding_matrix[i] = embedding_vector

   document_embeddings=np.zeros((len(tokenized_paded_documents1),100))
   dict_keys=list(tokenizer.word_index.keys())
   dict_values=list(tokenizer.word_index.values())
   for i in range(len(tokenized_paded_documents1)):
      joh1 = np.zeros((1,100),  dtype = np.float32)
      for j in range(len(tokenized_paded_documents1[0])):
            print(tokenized_paded_documents1[i][j])
            if tokenized_paded_documents1[i][j] != 0: 
                if tokenized_paded_documents1[i][j] in dict_values : # looking the word in trained vocab
                   val_index=dict_values.index(tokenized_paded_documents1[i][j])
                   print("Associated key is:")
                   myKey=dict_keys[val_index]
                   print(myKey)
                   #checking key lies in trained embeddings or not
                   if myKey in surbhi12.keys():
                      joh1 += surbhi12[myKey]
               
                   
      document_embeddings[i]=joh1
   for i in range(0,len(document_embeddings)):
     for j in range(0,len(document_embeddings[i])):
           if(math.isnan(document_embeddings[i][j])):
                      document_embeddings[i][j]=0;        
 
  
   cv = MLP_model.predict_proba(document_embeddings)
   print(cv)
   predicted_values = MLP_model.predict(document_embeddings)
   print(confusion_matrix(label13,predicted_values))
   print(classification_report(label13,predicted_values))  
   df3 = pd.DataFrame()
   df3["prob"]=cv[:,0]   
   df3["output"]=predicted_values
   df3.to_csv('testmlp.csv', sep='\t', index=False)      
   

In [20]:
test_MLP_model("/content/testset-levela.tsv", wv, iu)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Streaming output truncated to the last 5000 lines.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4710
Associated key is:
waging
672
Associated key is:
system
702
Associated key is:
super
81
Associated key is:
work
331
Associated key is:
deal
4711
Associated key is:
awards
33
Associated key is:
still
4712
Associated key is:
golden
4713
Associated key is:
pivot
6
Associated key is:
maga
1611
Associated key is:
argue
4714
Associated key is:
mail
1530
Associated key is:
felt
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1224
Associated key is:
david
11
Associated key is:
one
319
Associated key is:
sad
1614
Associated key is:
arrested
4715
Associated key is:
repeating
4716
Associated key is:
hosting
196
Associated key is:
yeah
19
Associated key is:
go
582
Associated key is:
college
383
Associated key is:
happened
4717
Associated key is:
achievements
7
Associated key is:
conservatives
21
Associated key is:
see
2
Asso

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
